## Data analysis

**FIXME: Diana, I have moved / rewritten most of the useful codes to the actual LMM analysis pipeline. Please salvage what you need here and try to build them to the pipeline -- after you complete MWE for fastGWA and SAIGE**

Pipeline to process the output files from the association results and analyze summary statistics

## Running sos notebook

On the Yale Farnam cluster,


On a local computer with for example 8 threads,

```
sos run ~/project/UKBB_GWAS_DEV/postprocessing.ipynb -q none -j 8
```

## Input files

### BOLT-LMM

The input files for this process are the ones created after running the `BoltLMM.ipynb` notebook that should be the following (example for INT-BMI):

* ukb_imp_chr{1:22}_v3.UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-BMI_withagesex_041720.boltlmm.snp_stats.gz (these are the association results for the bgen genotypes)
* ukb_imp_chr{1:22}_v3.UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-BMI_withagesex_041720.boltlmm.stats.gz (take into account that the 22 files generated in this step will have the same information, that is the association results for the hard called genotypes - PLINK genotypes)
* ukb_imp_chr{1:22}_v3.UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-BMI_withagesex_041720.boltlmm.snp_stats.stderr
* ukb_imp_chr{1:22}_v3.UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-BMI_withagesex_041720.boltlmm.snp_stats.stdout

### FastGWA

The input files for this process are the ones created after running the `fastGWA.ipynb` notebook and should be the following:

* bgenFile_chr{1:22}.TraitName.fastGWA (these are the association results for the bgen genotypes)
* bgenFile_chr{1:22}.TraitName.log


### Step 1: Merge the summary statistics generated during the association analysis of the imputed variants

In [92]:
# Merge all the bgen summary statistics
[merge_imputed]
input: snp_stats_file
output: f'{cwd}/{bgen_sumstats}.gz'
python: expand=True
    import gzip
    with gzip.open ('{_output}', 'wt') as outfile:
        with gzip.open({_input[0]:r}) as f:
            for line in f:
                outfile.write(line.decode('utf-8'))
            for files in [{_input:r,}][1:]:
                with gzip.open(files) as f:
                    for line in f:
                        if not line.startswith(b'SNP'):
                            outfile.write(line.decode('utf-8'))
    sum(1 for i in gzip.open('{_output}', 'r'))

Awk to find the minimum and maximum values in a specific column

In [ ]:
#awk '(NR==1){Min=$3;Max=$3};(NR>=3){if(Min>$3) Min=$3;if(Max<$3) Max=$3} END {printf "The Min is %d ,Max is %d",Min,Max}' ukb_mfi_chr1_v3.txt

Show the lowest p-values

In [ ]:
#gzcat Test_INT-BMI.txt.gz | awk -F "\t" '{ if ($16 <= 5e-8) { print } }'| sort -g | head -n 5

To find the overlapping variants between dataset 1 and dataset 2

In [ ]:
#awk 'NR==FNR {FILE1[$1]=$0; next} ($1 in FILE1) {print $0}' dataset2 dataset1 > dataset1_subset_overlapping_dataset2

The output file .clumped will contain the index SNPs for each LD block in column 3. To extract this column and create a vector to use for highlighting in the Manhattan Plot

```
awk '{ print $3 }' \
Test_INT-BMI.txt.clumped \
> dataset1_subset_overlapping_dataset2_clump1.SNPs
```

### Output some summary statistics from the phenotypic data
* Number of cases
* Number of controls

In [ ]:
[phenotype_summary]
input: phenoFile
output: None
bash: expand=True
    # Number of controls
    awk '{ if ($5 == 0) { print }' {_input} | wc -l 
    # Number of cases
    awk '{ if ($5 == 1) { print }' {_input} | wc -l